In [ ]:
import os
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from openpyxl import load_workbook

from scipy import stats as stats
from copy import copy as copy



In [ ]:
from functions.masterdata import master_data

In [ ]:
# read in paths from config file

configDict = {
    'rootDir': '',
    'initialDataPath' : '',
    'QCDataPath' : '',
    # 'labWorksheet01Path':'',
    'projectName':''
}

# with open(os.path.join(workingDir, 'config.txt'),'r') as f:
with open('config.txt','r') as f:
    lines = f.readlines()
    for line in lines:
        if not line.startswith('#'):
            line = line.strip()
            fields = line.split(':')

            if fields[0].strip()=='initialDataPath':
                configDict[fields[0].strip()] = fields[1].strip().strip('\'')
            else:
                configDict[fields[0].strip()] = fields[1].strip().strip('\'')
## ToDo: Add checks to ensure that minimal fields have been populated. Raise errors or warnings


In [ ]:
from openpyxl import load_workbook
import numpy as np
import pandas as pd

class master_data:
    def __init__(self, dataPath):
        ### import data from excel workbook
        
        self.wb = load_workbook(dataPath)
        self.segWs = self.wb['SegmentProperties']
        self.cntWs = self.wb['TargetCountMatrix']
        self.targWs = self.wb['TargetProperties']
        
        self.segValues = [[y.value for y in x] for x in self.segWs[self.segWs.calculate_dimension()]]
        self.cntValues = [[y.value for y in x] for x in self.cntWs[self.cntWs.calculate_dimension()]]
        self.targValues = [[y.value for y in x] for x in self.targWs[self.targWs.calculate_dimension()]]

        self.dropData = False
        
        
    def get_data(self):
        ### Convert nested list to a pandas dataFrame and extract expression data with labels
        # cntData = self.cntValues
        cntCols = self.cntValues[0]
        # df = pd.DataFrame(self.cntValues)
        cntIndex = [x[0] for x in self.cntValues[1:]]
        cntDF = pd.DataFrame(self.cntValues[1:], index=cntIndex, columns=cntCols)
        self.counts = cntDF.iloc[:,1:]
        self.counts = self.counts.astype(np.float64)      # Convert datatype to float64
        
        targCols = self.targValues[0]
        targIndex = [x[0] for x in self.targValues[1:]]
        targDF = pd.DataFrame(self.targValues[1:], index=targIndex, columns=targCols)
        # self.priobeInfo = targDF.iloc[:,1:]
        self.probeInfo = targDF
        
        segCols = self.segValues[0]
        segIndex = [x[4].replace(' | ',('_')) for x in self.segValues[1:]]
        self.segData = pd.DataFrame(self.segValues[1:], index=segIndex, columns=segCols)

        self.dataOrig = self.counts.copy()
        self.dataLog1 = np.log2(self.counts+1)            # Log transform data for QC and analysis steps

        self.probeClass = False                           # Keep a copy of the original data before transformation or normalisation
        # self.probeClass = df.iloc[self.targIdx:,2]      ### Index needs updating here also
        # self.probeClass.rename(index=rowLabels, inplace=True)
        # self.probeClass.rename(index='ProbeClass', inplace=True)

        # ToDo: Need to update probeclass handling and labeling
        
        self.probeClassDict = {
            'Positive': 'A',
            'Negative': 'B',
            'Control': 'C',
            'Endogenous': 'E'}

        return(self.counts, self.probeInfo, self.segData)

    def get_descriptors(self):
        pass

    def add_class_mean(self, df):
        pass

    def drop_AOIs(self, includes, writeOrig=False):
        pass
        
    def set_threshold(self, threshold):
        self.threshold = threshold
        # ToDo: Check that all values in master data are also included in threshold dataFrame
        # ToDo: Convert threshold data to 0/1 data if needed

    def drop_probes(self, labels):
        pass
        


In [ ]:
dataPath = os.path.join(configDict['rootDir'],configDict['initialDataPath'])
dataPath = os.path.join(configDict['rootDir'],configDict['QCDataPath'])

masterData = master_data(dataPath)


counts, probes, segs = masterData.get_data()


counts

# probes

# segs

In [ ]:
counts.index.name = 'TargetName'

In [ ]:
newIndex = [x for x in counts.index if not x=="NegProbe-WTX"]

In [ ]:
# Remove NegProbe-WTX
counts = counts.loc[newIndex]

In [ ]:
# Export counts to countFile.tsv

In [ ]:
counts.to_csv(os.path.join(configDict['rootDir'], 'countFile.tsv'), sep='\t')


In [ ]:
probes

In [ ]:
featureAnno = probes.loc[:,['TargetName','CodeClass']]

featureAnno['Negative'] = [ 'TRUE' if x == 'Negative' else 'False' for x in featureAnno['CodeClass']]

featureAnno.drop(labels = ['CodeClass'], axis=1, inplace = True)

featureAnno

In [ ]:
featureAnno = featureAnno.loc[newIndex]

In [ ]:
featureAnno['RTS_ID'] = [ 'RTS' + x for x in featureAnno['TargetName']]


featureAnno.reset_index(inplace = True)

featureAnno.set_index('RTS_ID', inplace = True)

In [ ]:
featureAnno

In [ ]:
# Export probe into to featureAnnoFile.tsv

In [ ]:
featureAnno.to_csv(os.path.join(configDict['rootDir'], 'featureAnnoFile.tsv'), sep='\t')

In [ ]:
segs.index.name='SlideName'

In [ ]:
segs.reset_index(drop=True)

In [ ]:
segs.columns

In [ ]:
# Rename Segment Labels to epithelial and Luminal (also cleans data)
segs.loc[segs['SegmentLabel'] == 'Full ROI','SegmentLabel'] = 'Epithelial'
segs.loc[segs['SegmentLabel'] == 'Segment 1','SegmentLabel'] = 'Epithelial'
segs.loc[segs['SegmentLabel'] == 'Segment 2','SegmentLabel'] = 'Luminal'


In [ ]:
segs['SlideName'] = [x.replace(' ','_') for x in segs['SlideName']]
segs['SlideName'] = [x.replace('-','_') for x in segs['SlideName']]
segs['SlideName'] = [x[-7:] for x in segs['SlideName']]

In [ ]:
segs['ScanLabel'] = [x.replace(' ','_') for x in segs['ScanLabel']]
segs['ScanLabel'] = [x.replace('-','_') for x in segs['ScanLabel']]
segs['ScanLabel'] = [x[-7:] for x in segs['ScanLabel']]

In [ ]:
segs.set_index('SlideName', inplace=True)

In [ ]:
# Export segs to sampleAnnoFile.tsv

In [ ]:
segs

In [ ]:
segs.to_csv(os.path.join(configDict['rootDir'], 'sampleAnnoFile.tsv'), sep='\t')


In [ ]:
# os.getcwd()

In [ ]:
# segsLuminal = segs[segs['SegmentLabel']=='Luminal']

# segsLuminal.to_csv(os.path.join(configDict['rootDir'], 'sampleAnnoFile_Luminal.tsv'), sep='\t')


# countsLuminal = counts[segsLuminal['SegmentDisplayName']]

# countsLuminal.to_csv(os.path.join(configDict['rootDir'], 'countFile_Luminal.tsv'), sep='\t')


In [ ]:
# configDict['rootDir']

In [ ]:
# segsEpithelial = segs[segs['SegmentLabel']=='Epithelial']

# segsEpithelial.to_csv(os.path.join(configDict['rootDir'], 'sampleAnnoFile_Epithelial.tsv'), sep='\t')


# countsEpithelial = counts[segsEpithelial['SegmentDisplayName']]

# countsEpithelial.to_csv(os.path.join(configDict['rootDir'], 'countFile_Epithelial.tsv'), sep='\t')
